# imports

In [ ]:
from tensorflow.keras import layers, optimizers, models, Sequential, initializers, constraints, regularizers, backend
import tensorflow as tf
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
%matplotlib
import seaborn as sns
sns.set_context('notebook',font_scale=1.15)

In [ ]:
tf.__version__

kernel contraint that effectively implements one-to-one connections

# define feature selection layer

In [ ]:
# all you need to create a mask matrix M, which is a NxN identity matrix
# and you can write a contraint like below
class DiagonalWeight(constraints.Constraint):
    """Constrains the weights to be diagonal.
    """
    def __call__(self, w):
        N = tf.shape(w)[-1]
        m = tf.eye(N)
        w = m*w
        return w

regularizer: L1 + time-dependent tanh

modified L0 loss: $\tilde L_0 = \alpha \sum_n |w_n+\beta|$, with $\beta=0.05$ here.

only one-to-one connections (=diagonal weight matrix): $w_{nk} = w_{n}\text{ if } $n=k$\text{ and 0 otherwise}$



In [ ]:
class L1_tilde(regularizers.Regularizer):
    """Regularizer for ...
    """

    def __init__(self, normalization = 1000, alpha = .05):
        self.counter = 0
        self.normalization = normalization
        self.alpha = alpha

    def __call__(self, x):
        regularization = 0.
        prefactor = min(1,self.counter/self.normalization)
        regularization += backend.sum(backend.abs(x+0.05))
        self.counter += 1
        return self.alpha*regularization

In [ ]:
class selection_layer(layers.Dense):
    def __init__(self, units, norm=1000, alpha=0.05):
        super(selection_layer, self).__init__(units, kernel_constraint=DiagonalWeight(),
                                        kernel_initializer = initializers.Ones(),
                                        kernel_regularizer= L1_tilde(alpha=alpha, normalization=norm),
                                        #bias_regularizer=constant_bias(),
                                        activation='relu',
                                        use_bias=False)

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs):
        weights = tf.linalg.tensor_diag_part((model.layers[0].weights[0]).numpy())
        weights_history.append(weights)

# test data

## most naive test

In [ ]:
model = Sequential([selection_layer(5)])
model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [ ]:
test_x = np.random.normal(size=(600,5))
test_y = np.concatenate((test_x[:,:4],np.random.normal(size = (600,1))), axis = -1)

In [ ]:
model.fit(test_x, test_y, epochs = 300, verbose = 1)

In [ ]:
model.layers[0].weights

### train for different alpha

In [ ]:
weights = np.array([])
r2s = np.array([])
for al in np.linspace(0.001, 2, 5):
    model = Sequential([selection_layer(units=X.shape[-1], alpha = al, norm=10000), layers.Dense(5, activation='relu'), layers.Dense(1)])
    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=500)
    weights = np.append(weights,tf.linalg.tensor_diag_part(model.layers[0].weights[0]).numpy())
    r2s = np.append(r2s,r2_score(y_test,model.predict(X_test)))
weights = weights.reshape(-1,13)

In [ ]:
#plt.figure()
ax = sns.heatmap((weights>0), cmap='Blues', cbar=False,)

In [ ]:
r2s

## classification data (sklearn dummy data)

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples = 1200, n_informative=10,n_repeated=0, n_classes=3, class_sep=1., n_clusters_per_class=2, shuffle=False)

In [ ]:
plt.figure()
plt.scatter(X[:, 1], X[:, 3],marker='o', c=y, s=25, edgecolor='k')
plt.show()

In [ ]:
#y = (y-y.mean())/y.std()
minmax = preprocessing.MinMaxScaler(feature_range=(0,10))
minmax.fit(X)
X = minmax.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
weights_history = []
loss = []
val_loss = []

num_alphas = 20
alpha_range = np.logspace(-2.4, 0.1, num_alphas)

weights = np.array([])
acc = np.array([])
for al in alpha_range:
    model = Sequential([selection_layer(units=X.shape[-1], alpha = al, norm=.5*100*int(800/32.)), layers.Dense(10, activation='relu'), layers.Dropout(.1), layers.Dense(3, activation='softmax')])
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    hi = model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=340, validation_split = 0.15, callbacks=[MyCallback()])
    weights = np.append(weights,tf.linalg.tensor_diag_part(model.layers[0].weights[0]).numpy())
    acc = np.append(acc, accuracy_score(y_test,model.predict_classes(X_test)))
    loss= np.append(loss, hi.history['loss'])
    val_loss = np.append(val_loss, hi.history['val_loss'])
weights = weights.reshape(-1,20)
weights_history = np.array(weights_history).reshape(num_alphas,-1,X.shape[-1])
loss = loss.reshape(num_alphas,-1)
val_loss = val_loss.reshape(num_alphas,-1)

In [ ]:
np.savez('classifier_for_poster', weight=weights, weights_history=weights_history, alpha_range=alpha_range, loss=loss, val_loss=val_loss, acc=acc)

In [ ]:
data = np.load('classifier_for_poster.npz')
weights, weights_history, alpha_range, loss, val_loss = data.f.weight, data.f.weights_history, data.f.alpha_range, data.f.loss, data.f.val_loss

In [ ]:
print(acc)

plot performance and selection

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,8))
#fig.subplots_adjust(hspace=0.5)
fig.suptitle(r'Performance and feature selection as function of $\alpha$')

ax[0].plot(alpha_range, acc, color = 'olivedrab')
#xlabel(r'regularization strength $\alpha$')
ax[0].set_ylabel(r'accuracy score')
ax[0].set_xscale('log')

tick = ticker.ScalarFormatter(useOffset=False, useMathText=True)
tick.set_powerlimits((0,0))
tg = [u"${}$".format(tick.format_data(round(x,3))) for x in alpha_range]

sns.heatmap((weights.T>0), ax = ax[1], cmap='tab20c_r', cbar=False, alpha=.7, linewidth=.5, xticklabels=tg)
ax[1].set_ylabel('feature #')
ax[1].set_xlabel(r'regularization strength $\alpha$')
ax[1].axhline(y=10, color = 'k', ls = ':')
plt.tight_layout()
plt.show()

weights

In [ ]:
plt.figure()
ind = -5
[plt.plot(x, c = 'indianred', lw = .75, alpha = .7) for x in weights_history[ind].T[:9]]
plt.plot(weights_history[ind].T[9], c = 'indianred', lw = .75, alpha = .7, label='informative features')
[plt.plot(x, c = 'dodgerblue', lw = .75, alpha = .7) for x in weights_history[ind].T[11:]]
plt.plot(weights_history[ind].T[10], c = 'dodgerblue', lw = .75, alpha = .7, label='non-informative features')
plt.xlabel('# batches')
plt.ylabel('selection layer weights')
plt.title(r'$\alpha$={:.2f}'.format(alpha_range[ind]))
plt.legend()
plt.tight_layout()
plt.show()

losses

In [ ]:
ind = -5
plt.figure()
plt.plot(loss[ind], label='loss')
plt.plot(val_loss[ind], label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('losses')
plt.title(r'$\alpha$={:.2f}'.format(alpha_range[ind]))
plt.legend()
plt.tight_layout()
plt.show()

### single $\alpha$ run

In [ ]:
weights_history = []
model = Sequential([selection_layer(units=X.shape[-1], alpha = 2., norm=100*int(800/32.)), layers.Dense(10, activation='relu'), layers.Dropout(.1), layers.Dense(3, activation='softmax')])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
history = model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=14, validation_split = 0.15, callbacks=[MyCallback()])

# hippocampus data

## load and prepare data

In [ ]:
%pylab

from scipy.io import loadmat
sys.path.append('/home/herfurtht/mpi-br/project1/')
sys.path.append('/home/herfurtht/mpi-br/rat/Neural_Decoding_fork/')

import preprocessing_funcs

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
mat = loadmat('hippo/data_CA1.mat')  # load mat-file
mat.keys()

In [ ]:
mat['time'] = arange(len(mat['run_speed']))/1000. #time in sec
mat['run_speed'], mat['x_pos'], mat['y_pos'] = mat['run_speed'].ravel(), mat['x_pos'].ravel(), mat['y_pos'].ravel()
mat['spikes'].shape, mat['x_pos'].reshape(-1,1).shape

In [ ]:
time_shift = 0 #here in ms

mat['spikes'] = mat['spikes'][:]
spike_times = preprocessing_funcs.binary_to_times(mat['spikes'], .001)
t_start = 0.
t_end = 595.- time_shift/1000.
vel_times = arange(0, 595., .001)
vels = array(list(zip(mat['x_pos'], mat['y_pos'])))

vels = vels[time_shift:]

In [ ]:
figure()
times = 0, 595000
scatter(mat['x_pos'][times[0]:times[1]], mat['y_pos'][times[0]:times[1]], c = arange(len(mat['x_pos'][times[0]:times[1]])), norm = mpl.colors.Normalize(vmin=0., vmax= len(mat['x_pos'][times[0]:times[1]])), cmap = cm.jet, s = .3)
show()

In [ ]:
dt= .05 #Size of time bins (in seconds)
downsample_factor=1 #Downsampling of output (to make binning go faster). 1 means no downsamplinga

In [ ]:
###Preprocessing to put spikes and output in bins###

#Bin neural data using "bin_spikes" function
neural_data= preprocessing_funcs.bin_spikes(spike_times,dt,t_start,t_end)
### remove neurons with too little spikes
neural_data = neural_data[:, neural_data.sum(0)> 10]

#Bin output (velocity) data using "bin_output" function
vels_binned= preprocessing_funcs.bin_output(vels,vel_times,dt,t_start,t_end,downsample_factor)

#velocities in either direction
#vels_binned = gradient(vels_binned, axis = 0)

In [ ]:
#6, 1, 6 before 0, 1, 21
bins_before= 5 #How many bins of neural data prior to the output are used for decoding
bins_current = 1 #Whether to use concurrent time bin of neural data
bins_after= 5 #How many bins of neural data after the output are used for decoding

In [ ]:
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
# Function to get the covariate matrix that includes spike history from previous bins
X=preprocessing_funcs.get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
#Put in "flat" format, so each "neuron / time" is a single feature
X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

In [ ]:
#Set decoding output
y=vels_binned

In [ ]:
X_train_mean, X_train, X_test, X_valid, X_flat_train_mean, X_flat_train, X_flat_test, X_flat_valid, y_train_mean, y_train, y_test, y_valid = preprocessing_funcs.get_training_data(X,y, [.8,.8], bins_before, bins_after)
X_test, y_test = X_valid, y_valid

## build decoder

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, SimpleRNN, GRU, Activation, Dropout, Conv1D, concatenate, Flatten, TimeDistributed
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
 #Declare model
model=Sequential() #Declare model
# Add selection layer (Time distributed)
model.add(TimeDistributed(selection_layer(units=X_train.shape[2], alpha = 5000, norm=100)))

#Add recurrent layer
model.add(LSTM(64, recurrent_dropout=.1,dropout=.1)) #Within recurrent layer, include dropout
model.add(Dropout(.1)) #Dropout some units (recurrent layer output units)

#Add dense connections to output layer
model.add(Dense(y_train.shape[1]))
#Fit model (and set fitting parameters)
model.compile(loss='mse',optimizer='rmsprop',metrics=['accuracy']) #Set loss function and optimizer
#Fit the model
model.fit(X_train,y_train, epochs=10,verbose=1, validation_split = .15, callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience= 2, verbose=0, mode='auto'), MyCallback()]) #Get predictions
y_valid_predicted_lstm=model.predict(X_valid)

#Get metric of fit
R2s_lstm=r2_score(y_valid,y_valid_predicted_lstm)

print('R2s:', R2s_lstm)

In [ ]:
%%time
weights_history = []
loss = []
val_loss = []

num_alphas = 12
alpha_range = np.logspace(-.3, 3.1, num_alphas)

weights = np.array([])
acc = np.array([])
for al in alpha_range:
    model=Sequential() #Declare model
    # Add selection layer (Time distributed)
    model.add(TimeDistributed(selection_layer(units=X_train.shape[2], alpha = al, norm=100)))

    #Add recurrent layer
    model.add(LSTM(64, recurrent_dropout=.1,dropout=.1)) #Within recurrent layer, include dropout
    model.add(Dropout(.1)) #Dropout some units (recurrent layer output units)

    #Add dense connections to output layer
    model.add(Dense(y_train.shape[1]))
    #Fit model (and set fitting parameters)
    model.compile(loss='mse',optimizer='rmsprop',metrics=['accuracy']) #Set loss function and optimizer
    #Fit the model
    hi = model.fit(X_train,y_train, epochs=25, verbose=1, validation_split = .15, 
                   callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience= 10, verbose=0, mode='auto'), MyCallback()]) 
    weights = np.append(weights,tf.linalg.tensor_diag_part(model.layers[0].weights[0]).numpy())
    acc = np.append(acc, r2_score(y_test,model.predict(X_test)))
    loss= np.append(loss, hi.history['loss'])
    val_loss = np.append(val_loss, hi.history['val_loss'])
weights = weights.reshape(-1,X_train.shape[2])
weights_history = np.array(weights_history).reshape(num_alphas,-1, X_train.shape[2])
loss = loss.reshape(num_alphas,-1)
val_loss = val_loss.reshape(num_alphas,-1)

save for later use

In [ ]:
np.savez('hippocampus_for_poster', weights=weights, weights_history=weights_history, alpha_range=alpha_range, loss=loss, val_loss=val_loss, acc=acc)

In [ ]:
data = np.load('hippocampus_for_poster.npz')
weights, weights_history, alpha_range, loss, val_loss = data.f.weight, data.f.weights_history, data.f.alpha_range, data.f.loss, data.f.val_loss

In [ ]:
acc

In [ ]:
plt.figure(figsize=(10,10))
ind = -2
[plt.plot(x, c = 'indianred', lw = .5) for x in weights_history[ind].T[:]]
#[plt.plot(x, c = 'dodgerblue') for x in weights_history[ind].T[10:]]
plt.tight_layout()
plt.show()

performance and selection

In [ ]:
alpha_new = array(['{:.2f}'.format(x) for x in alpha_range])
alpha_new = alpha_new.astype(float)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,8))
#fig.subplots_adjust(hspace=0.5)
fig.suptitle(r'Performance and feature selection as function of $\alpha$')

ax[0].plot(alpha_range, acc, color = 'olivedrab')
#xlabel(r'regularization strength $\alpha$')
ax[0].set_ylabel(r'$r^2$-score')
ax[0].set_xscale('log')

tick = ticker.ScalarFormatter(useOffset=False, useMathText=True)
tick.set_powerlimits((0,0))
tg = [u"${}$".format(tick.format_data(x)) for x in alpha_new]

sns.heatmap((weights[:,argsort(sum(weights>0, axis=0))].T>0), ax=ax[1],cmap='tab20c_r', alpha=.7, cbar=False, linewidth=.3, linecolor = 'k', xticklabels=tg)
ax[1].set_ylabel('cell # (sorted)')
ax[1].set_xlabel(r'regularization strength $\alpha$')
plt.tight_layout()
plt.show()

weights

In [ ]:
plt.figure()
ind = -2
[plt.plot(x, c = 'dodgerblue', lw = .75, alpha = .7) for x in weights_history[ind].T]
plt.xlabel('# batches')
plt.ylabel('selection layer weights')
plt.title(r'$\alpha$={:.2f}'.format(alpha_range[ind]))
plt.tight_layout()
plt.show()

losses

In [ ]:
ind = -2
plt.figure()
plt.plot(loss[ind], label='loss')
plt.plot(val_loss[ind], label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('losses')
plt.title(r'$\alpha$={:.2f}'.format(alpha_range[ind]))
plt.legend()
plt.tight_layout()
plt.show()

**notes**
- schedulder for offset (!)
- compare different feature selectors (and performance at random set with same size)
- show correlations
- hippocampus temporal data
- better for removing unimportant than for ranking!
- only for positive values